In [12]:
# Load from parent directory if not installed
import importlib
import os

if not importlib.util.find_spec("sammo"):
    import sys

    sys.path.append("../")
os.environ["CACHE_FILE"] = "cache/chaining_prompts.tsv"

# Using components to write metaprompts

In this tutorial, we'll visit a few more advanced concepts for building metaprompts with `SAMMO`. 

As mentioned before, metaprompts are essentially call graphs that are evaluated lazily with `.run*` methods. What they offer is a way to tell `SAMMO` which things depend on each other which in turns enables efficient scheduling.

In [13]:
# %load _init.py
import pathlib
import sammo
from sammo.runners import OpenAIChat
from sammo.base import Template
from sammo.components import Output, GenerateText, ForEach, Union
from sammo.extractors import ExtractRegex

API_CONFIG_FILE = pathlib.Path().cwd().parent / "config" / "personal.openai"
API_CONFIG = ""
if API_CONFIG_FILE.exists():
    API_CONFIG = API_CONFIG_FILE
if not API_CONFIG:
    raise ValueError('Please set API_CONFIG to {"api_key": "YOUR_KEY"}')

_ = sammo.setup_logger("WARNING")  # we're only interested in warnings for now

runner = OpenAIChat(
    model_id="gpt-3.5-turbo-16k",
    api_config=API_CONFIG,
    cache=CACHE_FILE,
    timeout=30,
)

## Passing on chat history

With newer chat-based LLMs, we might want to construct metaprompts that build on the chat history from previous steps.

In [14]:
first = GenerateText(
    "Hello! My name is Peter and I like horses.",
    system_prompt="Talk like Shakespeare.",
)

second = GenerateText("Write two sentences about my favorite animal.", history=first)
print(Output(second).run(runner))

third = GenerateText("Make it a short poem.", history=second)
print(Output(third).run(runner))

+---------+-------------------------------------------------------------+
| input   | output                                                      |
+=========+=============================================================+
| None    | Ah, thy favorite animal, the horse, doth possess a beauty   |
|         | unmatched by any other. With flowing mane and fiery spirit, |
|         | it doth gallop upon the fields, a symbol of freedom and     |
|         | untamed might.                                              |
+---------+-------------------------------------------------------------+
Constants: None
+---------+--------------------------------------------------------------+
| input   | output                                                       |
+=========+==============================================================+
| None    | In fields of green, the horse doth roam, Its noble spirit, a |
|         | beauty to behold. With flowing mane and hooves that pound,   |
|         | It ga

## Iterating over items
There are two main ways of iterating over sequences in `SAMMO`:

1. Sequence is known before prompt execution:
   1. Automatic iteration via `.run_on_datatable()` or `.run_on_dicts()` as we have seen in the quickstart example. This is recommended to annotate larger data in input-output pairs.
   2. Manual iteration via Python list and loop operators.
2. Sequence is the result of a prompt execution: Here, `SAMMO`provides the `ForEach` that is called lazily.

We now demonstrate the two bottom methods after loading our environment:

### Known sequence: Manual iteration via Python 

A common use case for this is when we want to repeat certain operations for a known number of times, e.g., sample LLM responses *N* times.

In [15]:
N = 5
fruits = [
    GenerateText("Generate the name of 1 fruit.", randomness=0.9, seed=i)
    for i in range(N)
]
Output(Union(*fruits)).run(runner)

+---------+-----------------------------------------------------+
| input   | output                                              |
+=========+=====================================================+
| None    | ['Mango', 'Mango', 'Mango', 'Pomegranate', 'Mango'] |
+---------+-----------------------------------------------------+
Constants: None

4 to 1 for Mango.

```{note}
We had to set `seed` to a different value in each `GenerateText` instance to disable local caching. Otherwise, we would get the same answer 5 times.
```


### Unknown sequence: Iterating via `ForEach`

Assume we want to generate a list of reasons for why someone might fail an exam and then for each reason, we want a possible idea to fix it.

In [7]:
fruits = ExtractRegex(
    GenerateText(
        "Generate a list of 5 fruits. Wrap each fruit with <item> and </item>."
    ),
    r"<item>(.*?)<.?item>"
)

fruit_blurbs = ForEach(
    "fruit",
    fruits,
    GenerateText(Template("Why is {{fruit}} a good fruit?")),
)
Output(fruit_blurbs).run(runner)

+---------+--------------------------------------------------------------+
| input   | output                                                       |
+=========+==============================================================+
| None    | ['Apple is considered a good fruit for several               |
|         | reasons:\n\n1. Nutritional Value: Apples are rich in         |
|         | essential nutrients like dietary fiber, vitamins             |
|         | (particularly vitamin C), and minerals (such as potassium).  |
|         | They are low in calories and fat, making them a healthy      |
|         | snack option.\n\n2. Antioxidants: Apples contain             |
|         | antioxidants, such as flavonoids and polyphenols, which help |
|         | protect the body against oxidative stress and inflammation.  |
|         | These antioxidants have been linked to various health        |
|         | benefits...                                                  |
+---------+--------------

Okay, that is more verbose than we want. What if we want to summarize it?

We simply add on another “layer”.

In [8]:
short_fruit_blurbs = ForEach(
    "reason",
    fruit_blurbs,
    GenerateText(
        Template(
            "Rewrite the following text to have less than 10 words.\n\nInput: {{reason}}\n\nOutput: "
        )
    ),
)
Output(short_fruit_blurbs).run(runner)

+---------+------------------------------------------------------------+
| input   | output                                                     |
+=========+============================================================+
| None    | ['Apple: Nutritious, antioxidant-rich, fibrous, hydrating, |
|         | versatile, long-lasting, tasty.', 'Oranges: Nutritious,    |
|         | antioxidant-rich, hydrating, aids digestion, versatile,    |
|         | refreshing.', 'Bananas: nutritious, energizing, aids       |
|         | digestion, heart-healthy, mood-enhancing, versatile.',     |
|         | 'Strawberries: low-cal, antioxidant-rich, high-fiber,      |
|         | delicious, versatile fruit.', 'Grapes: nutritious,         |
|         | hydrating, fibrous, heart-healthy, versatile, easy to      |
|         | incorporate.']                                             |
+---------+------------------------------------------------------------+
Constants: None

Nice! Alternatively, we could have tried changing the instructions to get this result directly, but LLMs often benefit from breaking tasks down into a series of smaller steps.

## Using a custom operator

Sometimes we would like to run a custom operation on the LLM output. The most flexibile solution would be to implement your own `Component` which we will cover in advanced concepts. In most cases, however, it is enough to simply use a `LambdaExtractor` with a user-defined function (UDF).

In [9]:
fruits_alt = ExtractRegex(
    GenerateText(
        "Generate a list of 5 fruits in alternating caps. Wrap each fruit with <item> and </item>."
    ),
    r"<item>(.*?)<.?item>"
)
Output(fruits_alt).run(runner)

+---------+--------------------------------------------------+
| input   | output                                           |
+=========+==================================================+
| None    | ['APPLE', 'banana', 'CHERRY', 'orange', 'GRAPE'] |
+---------+--------------------------------------------------+
Constants: None

Say we want to ensure now that each fruit name is in lowercase. To use the `LambdaExtractor`, we need to define a lambda function in a string (this is to make the whole prompt serializable). The lambda function takes one argument -- a single value passed on from the previous call and outputs a new value.

In [10]:
from sammo.extractors import LambdaExtractor

fruits_lowercased = LambdaExtractor(fruits_alt, "lambda x: x.lower()")
Output(fruits_lowercased).run(runner)

+---------+--------------------------------------------------+
| input   | output                                           |
+=========+==================================================+
| None    | ['apple', 'banana', 'cherry', 'orange', 'grape'] |
+---------+--------------------------------------------------+
Constants: None